In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install sentence-transformers
!pip install pinecone


In [3]:
!winget install Python.Python.3.11


^C


### Config things

In [12]:
import os

# Получаем текущий рабочий каталог
current_dir = os.getcwd()

# Строим путь на три уровня вверх
config_path = os.path.join(current_dir, '..', '..', 'llama.config')

# Нормализуем путь (убираем лишние '..' и т.д.)
config_path = os.path.normpath(config_path)


# Читаем файл
with open(config_path, 'r') as file:
    api_base = file.readline().strip()
    my_model = file.readline().strip()
    API_KEY = file.readline().strip()
    YOUR_API_KEY = file.readline().strip()
#    index_name = file.readline().strip()
index_name = 'red-llama-bertopic'

# Выводим значения
#print(f"API Base: {api_base}")
#print(f"My Model: {my_model}")
#print(f"API Key: {API_KEY}")
#print(f"Pinecone API Key: {YOUR_API_KEY}")
#print(f"Pinecone index name: {index_name}")


### Мультиязычная модель. Обучена!

```Python
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired

# Используем модель, поддерживающую русский язык
#sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
sentence_model = SentenceTransformer('distiluse-base-multilingual-cased')

# Мои настройки
my_nr_topics = 14 # Количество тем, которые вы хотите извлечь. Можете настроить это значение, чтобы контролировать количество тем.
my_nr_repr_docs=7 # Количество документов, используемых для представления каждой темы
my_min_topic_size=10  # Минимальный размер темы, сколько документов должно быть в теме, чтобы она считалась значимой
му_top_n_words=10 # Количество слов, которые вы хотите получить для каждой темы

# Настройка модели представления
representation_model = KeyBERTInspired(nr_repr_docs=my_nr_repr_docs, top_n_words=му_top_n_words)

# Создание модели BERTopic с настройками
topic_model = BERTopic(
    embedding_model=sentence_model,
    representation_model=representation_model,
    nr_topics=my_nr_topics,
    min_topic_size=my_min_topic_size,  # Минимальный размер темы
    calculate_probabilities=True,  # Вычисление вероятностей
    verbose=True  # Вывод дополнительной информации
)

# Обучение модели
#topics, probs = topic_model.fit_transform(docs)

# Сохранение модели
#topic_model.save("red_bertopic_model")

# Загрузка модели
from bertopic import BERTopic
loaded_model = BERTopic.load("red_bertopic_model")

# Добавление тем и вероятностей в DataFrame
#chunks_df['topic'] = topics
#chunks_df['probs'] = probs


# Вывод информации
print(chunks_df.columns)
print(chunks_df.info())
```

## Загрузка модели в оперативку с Yandex Cloud

In [5]:
# Импорт библиотек
import requests, joblib, io
import pandas as pd
from bertopic import BERTopic

# URL вашей модели
model_url = 'https://storage.yandexcloud.net/cyrilos/red_bertopic_model'



# Загрузка модели
response = requests.get(model_url)

# Проверка успешности запроса
if response.status_code == 200:
    # Загрузка модели из байтового потока
    loaded_model = joblib.load(io.BytesIO(response.content))
    print("Модель успешно загружена в память.")
else:
    print(f"Ошибка загрузки: {response.status_code}")

Модель успешно загружена в память.


## Настройки при обучении

In [13]:
# Мои настройки
my_nr_topics = 14 # Количество тем, которые вы хотите извлечь. Можете настроить это значение, чтобы контролировать количество тем.
my_nr_repr_docs=7 # Количество документов, используемых для представления каждой темы
my_min_topic_size=10  # Минимальный размер темы, сколько документов должно быть в теме, чтобы она считалась значимой
му_top_n_words=10 # Количество слов, которые вы хотите получить для каждой темы


In [14]:
# Получаем информацию по темам
topic_model = loaded_model
print(f"Настройки. Количество тем: {my_nr_topics}, Количество представи. документов: {my_nr_repr_docs}, Мин. размер: {my_min_topic_size}, Количество топ-слов: {му_top_n_words}")
topic_info = topic_model.get_topic_info()
print(topic_info)


Настройки. Количество тем: 14, Количество представи. документов: 7, Мин. размер: 10, Количество топ-слов: 10
   Topic  Count                                        Name  \
0     -1     21  -1_бакалавриат_бакалавр_диплом_аспирантура   
1      0     59     0_французский_бульдог_собака_популярный   
2      1     22     1_red_mad_robot_red_цифровой_разработка   
3      2     54            2_telegram_messages_message_chat   
4      3     62                 3_кожа_кислота_витамин_вода   
5      4     48         4_мост_бетон_строительство_строение   
6      5     19      5_звезда_ресторан_трёхзвёздочный_кухня   
7      6     50   6_работодатель_трудовой_работник_зарплата   
8      7     14              7_class_тренировка_клуб_тренер   
9      8     31       8_платёж_фискальный_оплачивать_оплата   

                                      Representation  \
0  [бакалавриат, бакалавр, диплом, аспирантура, о...   
1  [французский, бульдог, собака, популярный, щен...   
2  [red_mad_robot, red, цифро

In [15]:
# Получаем ключевые слова по темам
print(f"Количество тем заказано: {my_nr_topics}, Количество представи. документов: {my_nr_repr_docs}, Мин. размер: {my_min_topic_size}, Количество топ-слов: {му_top_n_words}")

for id in range(-1, len(topic_info)-1):
    topic_words = topic_model.get_topic(id)
    print(f"Ключевые слова по теме {id}: {topic_words}")


Количество тем заказано: 14, Количество представи. документов: 7, Мин. размер: 10, Количество топ-слов: 10
Ключевые слова по теме -1: [('бакалавриат', np.float32(0.31518373)), ('бакалавр', np.float32(0.27087903)), ('диплом', np.float32(0.2588203)), ('аспирантура', np.float32(0.23891361)), ('образование', np.float32(0.23076478)), ('студент', np.float32(0.19909294)), ('учебный', np.float32(0.18908706)), ('специальность', np.float32(0.18783751)), ('обучение', np.float32(0.18729722)), ('специалитета', np.float32(0.10582714))]
Ключевые слова по теме 0: [('французский', np.float32(0.47135442)), ('бульдог', np.float32(0.15758193)), ('собака', np.float32(0.14411372)), ('популярный', np.float32(0.1424608)), ('щенок', np.float32(0.12345127)), ('порода', np.float32(0.09492709)), ('животное', np.float32(0.0780914)), ('умный', np.float32(0.07168501)), ('дружелюбный', np.float32(0.05522666)), ('компактный', np.float32(0.04894419))]
Ключевые слова по теме 1: [('red_mad_robot', np.float32(0.47562537))

#### **Названия тем:**

- -1: Социальное обеспечение
- 0: Французский бульдог
- 1: rmr (red_mad_robot)
- 2: Telegram
- 3: Уход за кожей
- 4: Мостостроительство
- 5: Michelin
- 6: Трудовой кодекс
- 7: World Class
- 8: Оплата услуг

In [9]:
topics = {
    -1: "Социальное обеспечение",
    0: "Французский бульдог",
    1: "rmr (red_mad_robot)",
    2: "Telegram",
    3: "Уход за кожей",
    4: "Мостостроительство",
    5: "Michelin",
    6: "Трудовой кодекс",
    7: "World Class",
    8: "Оплата услуг"
}

# Чтение словаря в цикле
for key, value in topics.items():
    print(f"{key}: {value}")

-1: Социальное обеспечение
0: Французский бульдог
1: rmr (red_mad_robot)
2: Telegram
3: Уход за кожей
4: Мостостроительство
5: Michelin
6: Трудовой кодекс
7: World Class
8: Оплата услуг


### Поиск по новой новой обученной модели red_bertopic_model

# Поиск в Pinecone c фильтрацией

### Классификация моделью с threshold

In [10]:
import pandas as pd
import numpy as np

# Пример данных для предсказания
docs = [
    "Правила оплаты труда наемного работника",
    "Где лучше заниматься спортом и фитнесом?",
    "Сколько учиться на бакалавриате?",
    "Как оплатить коммунальные услуги?",
    "Когда была образована компания red mad robot?"
]

# Использование модели для предсказания тем
topics, probs = loaded_model.transform(docs)

# Создание DataFrame для хранения результатов
docs_df = pd.DataFrame(docs, columns=['document'])

# Установка порога вероятности
threshold = 0.17  # Установите желаемый порог


# Обработка вероятностей и определение тем
if probs.ndim > 1:  # Если probs двумерный
    print(f'Многомерный: {len(probs)}')
    # Создаем DataFrame для вероятностей
    probs_df = pd.DataFrame(probs, columns=[f'Topic_{i}' for i in range(probs.shape[1])])
    
    # Определяем темы на основе порога
    max_probs = probs_df.max(axis=1)  # Находим максимальные вероятности
    topics = np.where(max_probs > threshold, probs_df.idxmax(axis=1).str.replace('Topic_', '').astype(int), -1)
    
    # Объединяем два DataFrame
    docs_df = pd.concat([docs_df, probs_df], axis=1)
else:  # Если probs одномерный
    docs_df['probs'] = probs
    docs_df['topic'] = np.where(probs > threshold, -1, topics)  # Присваиваем -1, если вероятность ниже порога

# Добавляем темы в DataFrame
docs_df['topic'] = topics

# Настройка отображения для вывода всех строк и столбцов
pd.set_option('display.max_rows', None)  # Выводить все строки
pd.set_option('display.max_columns', None)  # Выводить все столбцы

# Вывод информации
print(docs_df)



Batches: 100%|██████████| 1/1 [00:00<00:00, 22.72it/s]


Многомерный: 5
                                        document   Topic_0   Topic_1  \
0        Правила оплаты труда наемного работника  0.001845  0.002625   
1       Где лучше заниматься спортом и фитнесом?  0.003845  0.007523   
2               Сколько учиться на бакалавриате?  0.000017  0.000103   
3              Как оплатить коммунальные услуги?  0.000921  0.001821   
4  Когда была образована компания red mad robot?  0.008871  0.344317   

    Topic_2   Topic_3   Topic_4   Topic_5   Topic_6   Topic_7   Topic_8  topic  
0  0.002814  0.001865  0.002320  0.005057  0.650599  0.006502  0.007198      6  
1  0.007848  0.003813  0.004663  0.011465  0.012265  0.466788  0.059668      7  
2  0.000102  0.000016  0.000020  0.023769  0.020645  0.027412  0.022548     -1  
3  0.002019  0.000917  0.001125  0.005306  0.012655  0.103519  0.165164     -1  
4  0.025852  0.008566  0.010118  0.016353  0.014130  0.018788  0.018950      1  


## Загрузка базы с Pinecone. Функция для запросов

In [16]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# Инициализация Pinecone
pc = Pinecone(api_key=YOUR_API_KEY)  # Замените на ваш API-ключ

# Подключение к индексу
index_name = "red-llama-bertopic"
index = pc.Index(index_name)

# Загрузка модели
model = SentenceTransformer('all-MiniLM-L6-v2')

def find_me(doc_n):
    # Индекс строки для анализа
    n = doc_n

    # Проверка, что индекс n находится в пределах DataFrame
    if n >= len(docs_df):
        print(f"Ошибка: индекс {n} выходит за пределы DataFrame.")
        exit()

    # Получение темы документа
    doc_topic = int(docs_df['topic'].iloc[n])

    # Фильтр для поиска в Pinecone
    filter_condition = {
        "topic": {
            "$eq": doc_topic
        }
    }

    # Обработка случая, когда тема не определена
#    if doc_topic == -1:
#        print("Тема не определена (вероятность ниже порога)")
#    else:
    my_topic = 'Topic_' + str(doc_topic)
    if my_topic in docs_df.columns:
        my_prob = docs_df.loc[n, my_topic]
    else:
        my_prob = "Тема не определена (вероятность ниже порога)"


    # Векторизация запроса
    query_text = docs_df['document'].iloc[n]
    query_embedding = model.encode(query_text)

    # Поиск в индексе
    results = index.query(
        vector=query_embedding.tolist(),  # Вектор запроса
        top_k=5,  # Количество возвращаемых результатов
        include_metadata=True,  # Включить метаданные в результаты
        filter=filter_condition  # Условия фильтрации
    )

    # Вывод результатов поиска
    print(f"Текст запроса: {query_text}\nСамая вероятная тема: {doc_topic}: {my_prob}")
    if not results['matches']:
        print("По вашему запросу ничего не найдено.")
    else:
        print("\nРезультаты поиска в Pinecone:")
        for match in results['matches']:
            print(f"ID: {match['id']}, Score: {match['score']}")
            if 'answer' in match['metadata']:
                print(f"Ответ: {match['metadata']['answer']}")
            if 'topic_name' in match['metadata']:
                print(f"Тема: {match['metadata']['topic_name']}")
            print("-" * 100)



### Вывод запросов по документам

In [17]:
find_me(0)

Текст запроса: Правила оплаты труда наемного работника
Самая вероятная тема: 6: 0.6505991507378555

Результаты поиска в Pinecone:
ID: chunk_324, Score: 0.74414736
Ответ: Предоставляются пособие по беременности и родам, единовременное пособие и ежемесячные выплаты по уходу за ребенком.
Тема: Трудовой кодекс
----------------------------------------------------------------------------------------------------
ID: chunk_316, Score: 0.731326461
Ответ: Работнику возмещаются расходы на проезд, проживание, суточные и сохраняется средняя зарплата.
Тема: Трудовой кодекс
----------------------------------------------------------------------------------------------------
ID: chunk_303, Score: 0.720413
Ответ: Работа в выходные и праздничные дни допускается только с согласия работника и оплачивается в двойном размере.
Тема: Трудовой кодекс
----------------------------------------------------------------------------------------------------
ID: chunk_329, Score: 0.718351
Ответ: Работник имеет право на 

In [18]:
find_me(1)

Текст запроса: Где лучше заниматься спортом и фитнесом?
Самая вероятная тема: 7: 0.46678813451419227

Результаты поиска в Pinecone:
ID: chunk_258, Score: 0.481509715
Ответ: Можно и нужно! World Class организует массу разных фитнес-туров как в России, так и за рубежом. Изучайте программы туров здесь: https://camp.worldclass.ru/ и отправляйтесь в фитнес-путешествие с World Class!
Тема: World Class
----------------------------------------------------------------------------------------------------
ID: chunk_260, Score: 0.460304111
Ответ: Вся информация представлена на сайте https://sport.worldclass.ru/. Также информацию о мероприятиях можно получить у тренеров в клубе.
Тема: World Class
----------------------------------------------------------------------------------------------------
ID: chunk_253, Score: 0.436094433
Ответ: В наших клубах есть возможность выбрать формат тренировок, исходя из любых целей и пожеланий. Подбор фитнес-программ по параметрам в клубах World Class на сайте http

In [19]:
find_me(2)

Текст запроса: Сколько учиться на бакалавриате?
Самая вероятная тема: -1: Тема не определена (вероятность ниже порога)

Результаты поиска в Pinecone:
ID: chunk_7, Score: 0.595531106
Ответ: **Как оплатить услуги бонусами от Дом.ру?**
	Участвуйте в программе Дом.ру Бонус. Так на вашем счёте будут копиться бонусы, которыми можно оплатить до 100% стоимости абонентской платы.​ 100 бонусов = 20 ₽. А ещё получайте скидки на услуги наших партнёров.
Тема: Социальное обеспечение
----------------------------------------------------------------------------------------------------
ID: chunk_33, Score: 0.585178196
Ответ: **Какие Минусы специалитета?**
	К минусам специалитета можно отнести более долгий срок обучения. Кроме того, возможность получить специальность есть не во всех вузах.
Тема: Социальное обеспечение
----------------------------------------------------------------------------------------------------
ID: chunk_39, Score: 0.526665151
Ответ: **Что будет с дипломами, полученным до введения 

In [20]:
find_me(3)

Текст запроса: Как оплатить коммунальные услуги?
Самая вероятная тема: -1: Тема не определена (вероятность ниже порога)

Результаты поиска в Pinecone:
ID: chunk_7, Score: 0.704367578
Ответ: **Как оплатить услуги бонусами от Дом.ру?**
	Участвуйте в программе Дом.ру Бонус. Так на вашем счёте будут копиться бонусы, которыми можно оплатить до 100% стоимости абонентской платы.​ 100 бонусов = 20 ₽. А ещё получайте скидки на услуги наших партнёров.
Тема: Социальное обеспечение
----------------------------------------------------------------------------------------------------
ID: chunk_33, Score: 0.615291953
Ответ: **Какие Минусы специалитета?**
	К минусам специалитета можно отнести более долгий срок обучения. Кроме того, возможность получить специальность есть не во всех вузах.
Тема: Социальное обеспечение
----------------------------------------------------------------------------------------------------
ID: chunk_8, Score: 0.544410229
Ответ: **Как копить бонусы Дом.ру?**
	При единовременно

In [21]:
find_me(4)

Текст запроса: Когда была образована компания red mad robot?
Самая вероятная тема: 1: 0.3443170428269253

Результаты поиска в Pinecone:
ID: chunk_223, Score: 0.856811285
Ответ: Компания red_mad_robot была основана в 2008 году.
Тема: rmr (red_mad_robot)
----------------------------------------------------------------------------------------------------
ID: chunk_224, Score: 0.82223171
Ответ: Изначально занималась разработкой мобильных игр для зарождающегося рынка мобильных приложений.
Тема: rmr (red_mad_robot)
----------------------------------------------------------------------------------------------------
ID: chunk_228, Score: 0.81879127
Ответ: red_mad_robot реализовала более 100 прорывных цифровых продуктов.
Тема: rmr (red_mad_robot)
----------------------------------------------------------------------------------------------------
ID: chunk_229, Score: 0.814380109
Ответ: В компании работает более 1050 экспертов.
Тема: rmr (red_mad_robot)
------------------------------------------

In [23]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
from config import config_me, path_me
# Импорт библиотек
import requests, joblib, io
import pandas as pd
from bertopic import BERTopic
from tqdm import tqdm

API_KEY, api_base, my_model, YOUR_API_KEY, index_name = config_me()


def init():
    # URL вашей модели
    model_url = 'https://storage.yandexcloud.net/cyrilos/red_bertopic_model'

    # Прогресс-бар для загрузки модели
    print("Загрузка модели...")
    response = requests.get(model_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 КБ
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

    # Загрузка модели из байтового потока
    loaded_model_bytes = io.BytesIO()
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        loaded_model_bytes.write(data)
    progress_bar.close()

    # Проверка успешности загрузки
    if response.status_code == 200:
        loaded_model = joblib.load(loaded_model_bytes)
        print("Модель успешно загружена в память.")
    else:
        print(f"Ошибка загрузки: {response.status_code}")
        return None, None, None

    # Прогресс-бар для инициализации Pinecone
    print("Инициализация Pinecone...")
    pc = Pinecone(api_key=YOUR_API_KEY)  # Замените на ваш API-ключ
    index_name = "red-llama-bertopic"
    index = pc.Index(index_name)

    # Прогресс-бар для загрузки модели векторизации
    print("Загрузка модели для векторизации...")
    model = SentenceTransformer('all-MiniLM-L6-v2')

    return loaded_model, index, model


'''
    # Пример данных для предсказания
    docs = [
        "Как работает машинное обучение?",
        "Какие есть книги по Python?",
        "Как настроить сервер на Linux?"
    ]

    # Использование модели для предсказания тем
    topics, probs = loaded_model.transform(docs)

    # Создание DataFrame для хранения результатов
    docs_df = pd.DataFrame(docs, columns=['document'])

    # Установка порога вероятности
    threshold = 0.17  # Установите желаемый порог

    # Обработка вероятностей и определение тем
    if probs.ndim > 1:  # Если probs двумерный
        print(f'Многомерный: {len(probs)}')
        # Создаем DataFrame для вероятностей
        probs_df = pd.DataFrame(probs, columns=[f'Topic_{i}' for i in range(probs.shape[1])])
        
        # Определяем темы на основе порога
        max_probs = probs_df.max(axis=1)  # Находим максимальные вероятности
        topics = np.where(max_probs > threshold, probs_df.idxmax(axis=1).str.replace('Topic_', '').astype(int), -1)
        
        # Объединяем два DataFrame
        docs_df = pd.concat([docs_df, probs_df], axis=1)
    else:  # Если probs одномерный
        docs_df['probs'] = probs
        docs_df['topic'] = np.where(probs > threshold, -1, topics)  # Присваиваем -1, если вероятность ниже порога

    # Добавляем темы в DataFrame
    docs_df['topic'] = topics
'''

# Функция для поиска в Pinecone
def find_in_pinecone(query_text, doc_topic, model, index):
    # Фильтр для поиска в Pinecone
    filter_condition = {
        "topic": {
            "$eq": doc_topic
        }
    }


    # Векторизация запроса
    query_embedding = model.encode(query_text)

    # Поиск в индексе
    results = index.query(
        vector=query_embedding.tolist(),  # Вектор запроса
        top_k=5,  # Количество возвращаемых результатов
        include_metadata=True,  # Включить метаданные в результаты
        filter=filter_condition  # Условия фильтрации
    )

    # Возвращаем результаты
    return results

### 2. **Интеграция результатов в запрос к чату**

threshold = 0.17  # Установите желаемый порог

def make_system_prompt(user_message, topic_model, model, index):
    # Классификация запроса пользователя
    topics, probs = topic_model.transform([user_message])
    #doc_topic = int(topics[0]) if probs[0] > threshold else -1
    doc_topic = int(topics[0]) if (probs[0] > threshold).any() else -1
    # Поиск в Pinecone
    results = find_in_pinecone(user_message, doc_topic, model, index)

    # Формирование контекста для запроса к модели
    context = "Результаты поиска:\n"
    if results['matches']:
        for match in results['matches']:
            if 'answer' in match['metadata']:
                context += f"- {match['metadata']['answer']}\n"
    else:
        context += "По вашему запросу ничего не найдено.\n"

    # Формирование системного запроса
    system_prompt = f"""
    Вот контекст, который может быть полезен для ответа на вопрос пользователя:
    {context}
    """
    return system_prompt

'''
    # Формирование финального запроса
    final_prompt = f"""
    {system_prompt}
    Вопрос пользователя: {user_message}
    """

    # Отправка запроса модели (например, OpenAI GPT)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Или другая модель
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )

    # Возвращаем ответ модели
    return response['choices'][0]['message']['content']
'''

'\n    # Формирование финального запроса\n    final_prompt = f"""\n    {system_prompt}\n    Вопрос пользователя: {user_message}\n    """\n\n    # Отправка запроса модели (например, OpenAI GPT)\n    response = openai.ChatCompletion.create(\n        model="gpt-3.5-turbo",  # Или другая модель\n        messages=[\n            {"role": "system", "content": system_prompt},\n            {"role": "user", "content": user_message}\n        ]\n    )\n\n    # Возвращаем ответ модели\n    return response[\'choices\'][0][\'message\'][\'content\']\n'

In [24]:
loaded_model, index, model = init()

Загрузка модели...


100%|██████████| 544M/544M [01:16<00:00, 7.10MiB/s] 


Модель успешно загружена в память.
Инициализация Pinecone...
Загрузка модели для векторизации...


In [5]:
user_message = "Правила оплаты труда наемного работника"
system_prompt = make_system_prompt(user_message, loaded_model, model, index)
print(system_prompt)

Batches: 100%|██████████| 1/1 [00:00<00:00, 35.71it/s]



    Вот контекст, который может быть полезен для ответа на вопрос пользователя:
    Результаты поиска:
- Предоставляются пособие по беременности и родам, единовременное пособие и ежемесячные выплаты по уходу за ребенком.
- Работнику возмещаются расходы на проезд, проживание, суточные и сохраняется средняя зарплата.
- Работа в выходные и праздничные дни допускается только с согласия работника и оплачивается в двойном размере.
- Работник имеет право на уведомление об изменениях за два месяца и на отказ от работы в случае несогласия.
- Работник может восстановиться на работе через суд, а также получить компенсацию за вынужденный прогул.

    


In [ ]:
from flask import Flask, request, jsonify
import requests
from io import BytesIO
from bertopic import BERTopic
import logging

app = Flask(__name__)

# URL модели BERTopic в Yandex Object Storage
model_url = 'https://storage.yandexcloud.net/cyrilos/red_bertopic_model'

# Загрузка модели BERTopic
def load_model():
    response = requests.get(model_url)
    if response.status_code == 200:
        model_data = BytesIO(response.content)
        model = BERTopic.load(model_data)  # Загружаем модель BERTopic
        return model
    else:
        raise Exception(f"Ошибка загрузки модели: {response.status_code}")

# Загружаем модель при старте приложения
model = load_model()

# Маршрут для предсказания тем
@app.route('/predict_topics', methods=['POST'])
def predict_topics():
    # Получаем данные из запроса
    data = request.json
    docs = data['docs']  # Ожидается список строк (документов)
    logging.info(f"Получены документы: {docs}")

    # Выполняем предсказание
    topics, probs = model.transform(docs)
    logging.info(f"Предсказанные темы: {topics}")
    logging.info(f"Вероятности: {probs}")

    # Возвращаем результат
    return jsonify({
        'topics': topics.tolist(),  # Преобразуем массив NumPy в список
        'probabilities': probs.tolist() if probs is not None else None
    })

# Запуск приложения
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: expected str, bytes or os.PathLike object, not BytesIO

In [3]:
from flask import Flask, request, jsonify
import requests
from io import BytesIO
from bertopic import BERTopic
import logging
from tqdm import tqdm

app = Flask(__name__)

# URL модели BERTopic в Yandex Object Storage
model_url = 'https://storage.yandexcloud.net/cyrilos/red_bertopic_model'

# Загрузка модели BERTopic с прогрессбаром
def load_model():
    response = requests.get(model_url, stream=True)
    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))
        model_data = BytesIO()
        with tqdm(total=total_size, unit='B', unit_scale=True, desc="Загрузка модели") as pbar:
            for chunk in response.iter_content(chunk_size=1024):
                model_data.write(chunk)
                pbar.update(len(chunk))
        model_data.seek(0)
        model = BERTopic.load(model_data)  # Загружаем модель BERTopic
        return model
    else:
        raise Exception(f"Ошибка загрузки модели: {response.status_code}")

# Загружаем модель при старте приложения
logging.info("Загрузка модели BERTopic...")
model = load_model()
logging.info("Модель BERTopic успешно загружена.")
'''
# Маршрут для предсказания тем
@app.route('/predict_topics', methods=['POST'])
def predict_topics():
    # Получаем данные из запроса
    data = request.json
    docs = data['docs']  # Ожидается список строк (документов)
    logging.info(f"Получены документы: {docs}")

    # Выполняем предсказание
    topics, probs = model.transform(docs)
    logging.info(f"Предсказанные темы: {topics}")
    logging.info(f"Вероятности: {probs}")

    # Возвращаем результат
    return jsonify({
        'topics': topics.tolist(),  # Преобразуем массив NumPy в список
        'probabilities': probs.tolist() if probs is not None else None
    })
'''
message = "Когда образована компания Red Mad Robot?"
# Маршрут для классификации сообщения пользователя
@app.route('/classify', methods=['POST'])
def classify():
    # Получаем данные из запроса
    data = request.json
    message = data['message']  # Ожидается строка (сообщение пользователя)
    logging.info(f"Получено сообщение: {message}")

    # Выполняем предсказание
    topic, prob = model.transform([message])
    logging.info(f"Предсказанная тема: {topic[0]}")
    logging.info(f"Вероятность: {prob[0] if prob is not None else None}")

    # Возвращаем результат
    return jsonify({
        'topic': int(topic[0]),  # Преобразуем в int
        'probability': float(prob[0]) if prob is not None else None
    })

# Запуск приложения
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    app.run(host='0.0.0.0', port=5000)


Загрузка модели: 100%|██████████| 544M/544M [01:23<00:00, 6.52MB/s] 


TypeError: expected str, bytes or os.PathLike object, not BytesIO

In [5]:
from flask import Flask, request, jsonify
import requests
from io import BytesIO
from bertopic import BERTopic
import logging
from tqdm import tqdm
import tempfile
import os

app = Flask(__name__)

# URL модели BERTopic в Yandex Object Storage
model_url = 'https://storage.yandexcloud.net/cyrilos/red_bertopic_model'

# Загрузка модели BERTopic с прогрессбаром
def load_model():
    response = requests.get(model_url, stream=True)
    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))
        model_data = BytesIO()
        with tqdm(total=total_size, unit='B', unit_scale=True, desc="Загрузка модели") as pbar:
            for chunk in response.iter_content(chunk_size=1024):
                model_data.write(chunk)
                pbar.update(len(chunk))
        model_data.seek(0)

        # Сохраняем данные во временный файл
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            tmp_file.write(model_data.read())
            tmp_file_path = tmp_file.name

        # Загружаем модель из временного файла
        model = BERTopic.load(tmp_file_path)

        # Удаляем временный файл
        os.unlink(tmp_file_path)

        return model
    else:
        raise Exception(f"Ошибка загрузки модели: {response.status_code}")

# Загружаем модель при старте приложения
logging.info("Загрузка модели BERTopic...")
model = load_model()
logging.info("Модель BERTopic успешно загружена.")

{
  "docs": [
    "Это первый документ.",
    "Это второй документ.",
    "Это третий документ."
  ]
}
import requests



# Маршрут для предсказания тем
@app.route('/predict_topics', methods=['POST'])
def predict_topics():
    # Получаем данные из запроса
    data = request.json
    docs = data['docs']  # Ожидается список строк (документов)
    logging.info(f"Получены документы: {docs}")

    # Выполняем предсказание
    topics, probs = model.transform(docs)
    logging.info(f"Предсказанные темы: {topics}")
    logging.info(f"Вероятности: {probs}")

    # Возвращаем результат
    return jsonify({
        'topics': topics.tolist(),  # Преобразуем массив NumPy в список
        'probabilities': probs.tolist() if probs is not None else None
    })

# Маршрут для классификации сообщения пользователя
@app.route('/classify', methods=['POST'])
def classify():
    # Получаем данные из запроса
    data = request.json
    message = data['message']  # Ожидается строка (сообщение пользователя)
    logging.info(f"Получено сообщение: {message}")

    # Выполняем предсказание
    topic, prob = model.transform([message])
    logging.info(f"Предсказанная тема: {topic[0]}")
    logging.info(f"Вероятность: {prob[0] if prob is not None else None}")

    # Возвращаем результат
    return jsonify({
        'topic': int(topic[0]),  # Преобразуем в int
        'probability': float(prob[0]) if prob is not None else None
    })


# URL сервера
url = 'http://localhost:5000/classify'

# Тело запроса
data = {
    "message": "Это пример сообщения для классификации."
}

# Отправка POST-запроса
response = requests.post(url, json=data)

# Вывод результата
print(response.json())


# Запуск приложения
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    app.run(host='0.0.0.0', port=5000)


Загрузка модели: 100%|██████████| 544M/544M [01:17<00:00, 7.06MB/s] 


ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /classify (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B22F885350>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))